## 재무제표 데이터를 지표화

In [1]:
import requests
from bs4 import BeautifulSoup
import io
import zipfile
import xmltodict
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np

## 재무상태표
1. 자본총계: 자산총계 - 부채총계

## 손익계산서
1. 매출액: 매출총이익 + 매출원가
2. PSR: 매출액 / 주가
3. GP/A: 매출총이익 / 자산총계


In [153]:
#2. PSR = 당기 공시기점의 종가 / (당기매출액 / 발행주식수) 

# corp_list랑 corp_code_list는 api_final_result 파일에서 가져오면 됨

api_final_result = pd.read_csv('./api_final_result.csv', dtype={'종목코드':str, '주가코드': str})

# 법인등록번호 가져오기
corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
corp_code_list = ['00126380', '00126256', '00164742', '00413046', '00155319']
crnos = []
crnos_dict = {}
for corp_code in corp_code_list:
    params_1 = {
        'crtfc_key' : '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85',
        'corp_code' : corp_code
    }
    r_temp = requests.get('https://opendart.fss.or.kr/api/company.json', params = params_1)
    import json
    crnos.append(json.loads(r_temp.text)['jurir_no'])
    
    crnos_dict[corp_code] = json.loads(r_temp.text)['jurir_no']


#가져온 법인등록번호로 발행주식수 가져오기

URL = "http://apis.data.go.kr/1160100/service/GetStocIssuInfoService/getItemBasiInfo"
psr_dict = {}

for crno in crnos:
    params = {
        'pageNo' : 1,
        'numOfRows' : 1,
        'resultType' : 'json',
        'serviceKey' : '+21XqtAVW3QkzmAlufEeZE+QJaIxraq0IUyEwMLo0KbKkVfrEecTdm2PLj3j4iNwjSWxzydNtkaFlYQluK68jA==',
        'crno' : crno
    }
    r = requests.get(URL, params = params)
    import json
    try: 
        json_obj = json.loads(r.text)['response']['body']['items']['item'][0]
        
        # 키 : 주식발행회사명, 값 : 발행주식수 -> psr_dict에 저장
        psr_dict[json_obj["stckIssuCmpyNm"]] = json_obj['issuStckCnt'] 
    except: 
        print(json_obj["stckIssuCmpyNm"])
        continue


# stock_data에서 날짜에 맞는 종가 가져오기
import datetime
stock_data = pd.read_csv('./stock_data(5y).csv', dtype={'Code':str})
stock_data = stock_data[['Date', 'Code', 'Close']]
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y%m%d')


# api_final_result에 stock_data의 기준일자에 맞는 종가를 붙이기
for i in range(len(api_final_result['기준일자'])):
    api_final_result.loc[i, '기준일자'] = api_final_result['기준일자'][i]//1000000


# 분기별 종가를 새 컬럼으로 붙임. 이름은 '분기종가'
api_final_result['분기종가'] = None

for i in range(len(api_final_result)):
    for j in range(len(stock_data)):
        if (str(api_final_result.loc[i, '기준일자']) == stock_data.loc[j, 'Date']) and (str(api_final_result.loc[i, '주가코드']) == stock_data.loc[j, 'Code']):
            api_final_result.loc[i, '분기종가'] = stock_data.loc[j, 'Close']
        else:
            continue


api_final_result #주가코드, 분기별 종가 추가 완료

#발행주식수 추가하자
crnos_list = list(crnos_dict.keys())

api_final_result['발행주식수'] = None

for i in range(len(api_final_result)):
    for j in range(len(crnos_dict.keys())):
        if (api_final_result.loc[i, '종목코드'] == crnos_list[j]):
            api_final_result.loc[i, '발행주식수'] = crnos_dict[crnos_list[j]]


#발행주식수 컬럼 추가 완료.

api_final_result

,Unnamed: 0,기준일자,기업명,종목코드,이연법인세부채,분기코드,비유동부채,비지배지분,유형자산의 처분,유동자산,...,유형자산,이자의 수취,매출총이익,부채총계,금융수익,사업년도,분기명,주가코드,분기종가,발행주식수
0,0,20180515,삼성전자,00126380,1.234999e+13,11013,1842051000000,94546000000,1.415690e+11,72370085000000,...,66737192000000,94118000000,17410718000000,49010792000000,799826000000,2018,1분기,005930,49200,1301110006246
1,1,20180814,삼성전자,00126380,1.318636e+13,11012,1926719000000,81090000000,1.982810e+11,68070224000000,...,69235302000000,222063000000,16708550000000,42130475000000,1290033000000,2018,2분기,005930,45150,1301110006246
2,2,20181114,삼성전자,00126380,1.375049e+13,11014,1771916000000,178867000000,2.829450e+11,81904347000000,...,69525464000000,360840000000,20067142000000,48140630000000,739900000000,2018,3분기,005930,44100,1301110006246
3,3,20190401,삼성전자,00126380,1.516252e+13,11011,2888179000000,450142000000,3.405580e+11,80039455000000,...,70602493000000,459074000000,68715364000000,46033232000000,3737494000000,2018,4분기,005930,45050,1301110006246
4,4,20190515,삼성전자,00126380,1.540458e+13,11013,2578260000000,31372000000,1.054600e+11,82311689000000,...,70498616000000,98290000000,10586172000000,46897440000000,643068000000,2019,1분기,005930,42550,1301110006246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,20220516,POSCO홀딩스,00155319,3.009177e+12,11013,4467709468629,214947479400,-1.448500e+10,5875723628951,...,143758772501,38674284562,2928765425577,6207723607323,-125555822774,2022,1분기,005490,281000,1746110000741
80,80,20220816,POSCO홀딩스,00155319,2.947065e+12,11012,4318126500164,254454407727,-1.448500e+10,4060603659405,...,143189169662,62434274729,2733139872051,4663745800378,-258483606915,2022,2분기,005490,258000,1746110000741
81,81,20221114,POSCO홀딩스,00155319,2.785763e+12,11014,4201026696794,213864027182,-1.448500e+10,3457736374418,...,142646703013,87251112870,1607959638506,4326104941212,-359096307742,2022,3분기,005490,294500,1746110000741
82,82,20230309,POSCO홀딩스,00155319,2.480379e+12,11011,3858263509655,413709215105,-1.444721e+10,4308290115146,...,145005572040,44614715351,7649292423855,3977256531610,-346941713822,2022,4분기,005490,322500,1746110000741


# 2. PSR = 당기 공시기점의 종가 / (당기매출액 / 발행주식수) 

# api_result_csv 에 새로운 컬럼으로 붙일 것. 

# 당기 종가 * 발행주식수 / 매출총액

분기종가 * 발행주식수 / (매출총이익 + 매출원가)

In [162]:
api_final_result['PSR'] = None

for i in range(len(api_final_result)):
    mca = int(api_final_result.loc[i, '매출총이익'] + api_final_result.loc[i, '매출원가'])*1000
    api_final_result.loc[i, 'PSR'] = int(api_final_result.loc[i, '분기종가']) * int(api_final_result.loc[i, '발행주식수']) / mca
    
    
api_final_result

,Unnamed: 0,기준일자,기업명,종목코드,이연법인세부채,분기코드,비유동부채,비지배지분,유형자산의 처분,유동자산,...,이자의 수취,매출총이익,부채총계,금융수익,사업년도,분기명,주가코드,분기종가,발행주식수,PSR
0,0,20180515,삼성전자,00126380,1.234999e+13,11013,1842051000000,94546000000,1.415690e+11,72370085000000,...,94118000000,17410718000000,49010792000000,799826000000,2018,1분기,005930,49200,1301110006246,1.502444
1,1,20180814,삼성전자,00126380,1.318636e+13,11012,1926719000000,81090000000,1.982810e+11,68070224000000,...,222063000000,16708550000000,42130475000000,1290033000000,2018,2분기,005930,45150,1301110006246,1.421893
2,2,20181114,삼성전자,00126380,1.375049e+13,11014,1771916000000,178867000000,2.829450e+11,81904347000000,...,360840000000,20067142000000,48140630000000,739900000000,2018,3분기,005930,44100,1301110006246,1.204293
3,3,20190401,삼성전자,00126380,1.516252e+13,11011,2888179000000,450142000000,3.405580e+11,80039455000000,...,459074000000,68715364000000,46033232000000,3737494000000,2018,4분기,005930,45050,1301110006246,0.344021
4,4,20190515,삼성전자,00126380,1.540458e+13,11013,2578260000000,31372000000,1.054600e+11,82311689000000,...,98290000000,10586172000000,46897440000000,643068000000,2019,1분기,005930,42550,1301110006246,1.494725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,20220516,POSCO홀딩스,00155319,3.009177e+12,11013,4467709468629,214947479400,-1.448500e+10,5875723628951,...,38674284562,2928765425577,6207723607323,-125555822774,2022,1분기,005490,281000,1746110000741,52.230744
80,80,20220816,POSCO홀딩스,00155319,2.947065e+12,11012,4318126500164,254454407727,-1.448500e+10,4060603659405,...,62434274729,2733139872051,4663745800378,-258483606915,2022,2분기,005490,258000,1746110000741,164.838879
81,81,20221114,POSCO홀딩스,00155319,2.785763e+12,11014,4201026696794,213864027182,-1.448500e+10,3457736374418,...,87251112870,1607959638506,4326104941212,-359096307742,2022,3분기,005490,294500,1746110000741,319.991546
82,82,20230309,POSCO홀딩스,00155319,2.480379e+12,11011,3858263509655,413709215105,-1.444721e+10,4308290115146,...,44614715351,7649292423855,3977256531610,-346941713822,2022,4분기,005490,322500,1746110000741,39.90214


In [165]:
# 망했다...gpa만 살리자

api_final_result['GP/A'] = None

for i in range(len(api_final_result)):
    gpa = int(api_final_result.loc[i, '매출총이익']) / int(api_final_result.loc[i, '자산총계'])
    api_final_result.loc[i, 'GP/A'] = gpa*100
    
    
api_final_result

,Unnamed: 0,기준일자,기업명,종목코드,이연법인세부채,분기코드,비유동부채,비지배지분,유형자산의 처분,유동자산,...,매출총이익,부채총계,금융수익,사업년도,분기명,주가코드,분기종가,발행주식수,PSR,GP/A
0,0,20180515,삼성전자,00126380,1.234999e+13,11013,1842051000000,94546000000,1.415690e+11,72370085000000,...,17410718000000,49010792000000,799826000000,2018,1분기,005930,49200,1301110006246,1.502444,8.4845
1,1,20180814,삼성전자,00126380,1.318636e+13,11012,1926719000000,81090000000,1.982810e+11,68070224000000,...,16708550000000,42130475000000,1290033000000,2018,2분기,005930,45150,1301110006246,1.421893,8.191666
2,2,20181114,삼성전자,00126380,1.375049e+13,11014,1771916000000,178867000000,2.829450e+11,81904347000000,...,20067142000000,48140630000000,739900000000,2018,3분기,005930,44100,1301110006246,1.204293,9.213587
3,3,20190401,삼성전자,00126380,1.516252e+13,11011,2888179000000,450142000000,3.405580e+11,80039455000000,...,68715364000000,46033232000000,3737494000000,2018,4분기,005930,45050,1301110006246,0.344021,31.373819
4,4,20190515,삼성전자,00126380,1.540458e+13,11013,2578260000000,31372000000,1.054600e+11,82311689000000,...,10586172000000,46897440000000,643068000000,2019,1분기,005930,42550,1301110006246,1.494725,4.797787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,20220516,POSCO홀딩스,00155319,3.009177e+12,11013,4467709468629,214947479400,-1.448500e+10,5875723628951,...,2928765425577,6207723607323,-125555822774,2022,1분기,005490,281000,1746110000741,52.230744,5.49138
80,80,20220816,POSCO홀딩스,00155319,2.947065e+12,11012,4318126500164,254454407727,-1.448500e+10,4060603659405,...,2733139872051,4663745800378,-258483606915,2022,2분기,005490,258000,1746110000741,164.838879,5.308793
81,81,20221114,POSCO홀딩스,00155319,2.785763e+12,11014,4201026696794,213864027182,-1.448500e+10,3457736374418,...,1607959638506,4326104941212,-359096307742,2022,3분기,005490,294500,1746110000741,319.991546,3.16151
82,82,20230309,POSCO홀딩스,00155319,2.480379e+12,11011,3858263509655,413709215105,-1.444721e+10,4308290115146,...,7649292423855,3977256531610,-346941713822,2022,4분기,005490,322500,1746110000741,39.90214,14.954886


In [166]:
api_final_result.to_csv('api_result_with_gpa.csv')